In [7]:
abstract MicroLang = {
    cat
    Utt ;
    
    S ;
    VP ;
    Comp ;
    AP ;
    CN ; -- common noun (without determiner)
    NP ; -- noun phrase
    Det ;
    Prep ;
    V ;
    V2 ;
    A ; 
    N ;
    Pron ;
    Adv ;

    fun
-- Phrase
    UttS    : S  -> Utt ;
    UttNP   : NP -> Utt ;

-- Sentence
    PredVPS : NP -> VP -> S ;

-- Verb
    UseV    : V  -> VP ;
    ComplV2 : V2 -> NP -> VP ;
    UseComp : Comp -> VP ;
    CompAP : AP -> Comp ;
    
-- Noun
    DetCN      : Det -> CN -> NP ;
    a_Det      : Det ;
    aPl_Det    : Det ;
    the_Det    : Det ;
    thePl_Det  : Det ;
    UseN       : N -> CN ;
    AdjCN      : AP -> CN -> CN ;
    
-- Adjective
    PositA     : A -> AP ;

    fun
    boy_N : N ;
    girl_N : N ;
    house_N : N ;
    clever_A : A ;
    big_A : A ;
    jump_V : V ;
    play_V : V ;
    love_V2 : V2 ;
    see_V2 : V2 ;
    
}


Defined MicroLang

In [8]:
resource MicroResSwe = {

    param
        Number = Sg | Pl ;
        Species = Indef | Def ;
        Gender = Utr | Neutr ;
        Degree = Positive | Comparative | Superlative ;
        
        VForm = Inf | Pres | Pret | Perf ;
    oper
    -- Noun
        Noun : Type = { s : Number => Species => Str ; g : Gender} ;

        mkNoun : Str -> Str -> Str -> Str -> Gender -> Noun
        = \sgIndef,sgDef,plIndef,plDef,gen -> {
          s = table {
              Sg => table {Indef => sgIndef ; Def => sgDef} ;
              Pl => table {Indef => plIndef; Def => plDef}
          } ;
          g = gen
        } ;

        smartNoun : Str -> Gender -> Noun = \sgIndef,gen -> case gen of {
        Utr => case sgIndef of {
            pojk + "e" => mkNoun sgIndef (pojk + "en") (pojk + "ar") (pojk + "arna") gen ;
            flick + "a" => mkNoun sgIndef (flick + "an") (flick + "or") (flick + "orna") gen ;
            cyk + "el" => mkNoun sgIndef (cyk + "eln") (cyk + "lar") (cyk + "larna") gen ;
            vä + "n" => mkNoun sgIndef (vä + "nnen") (vä + "nner") (vä + "nnerna") gen ;
            dat + "or" => mkNoun sgIndef (dat + "orn") (dat + "orer") (dat + "orerna") gen ;
            _ => mkNoun sgIndef (sgIndef + "en") (sgIndef + "ar") (sgIndef + "arna") gen
            } ;
        Neutr => case sgIndef of {
            äppl + "e" => mkNoun sgIndef (äppl + "et")  (äppl + "en") (äppl + "ena") gen ;
            vatt + "en" => mkNoun sgIndef (vatt + "net")  (vatt + "en") (vatt + "nen") gen ;
            _ => mkNoun sgIndef (sgIndef + "et") (sgIndef) (sgIndef + "en") gen
            }
        } ;
    
    -- Adjective
        Adjective : Type = {s : Number => Species => Gender => Str} ;
        
        mkAdjective : Str -> Str -> Str -> Str -> Adjective
        = \utrSgIndef, neutrSgIndef, sgDef, pl -> {
          s = table {
              Sg => table {
                  Indef => table {Utr => utrSgIndef ; Neutr => neutrSgIndef}; 
                  Def => table {_ => sgDef}
              } ;
              Pl => table {
                    _=> table {
                        _ => pl
                    }
                }
            }
        } ;
 
        smartAdjective : Str -> Adjective = \adj -> case adj of {
            smar + "t" => mkAdjective adj adj (adj + "a") (adj + "a") ;
            _ => mkAdjective adj (adj + "t") (adj + "a") (adj + "a")
        } ;
  
    -- Verb
        Verb : Type = {s : VForm => Str} ;

        mkVerb : (inf,pres,pret,perf : Str) -> Verb
            = \inf,pres,pret,perf -> {
            s = table {
                Inf => inf ;
                Pres => pres ;
                Pret => pret ;
                Perf => perf
            }
        } ;

        regVerb : (inf : Str) -> Verb = \inf ->
            mkVerb inf (inf + "r") (inf + "de") (inf + "t") ;

        irregVerb : (inf,pres,pret,perf : Str) -> Verb = 
            \inf,pres,pret,perf ->
                mkVerb inf pres pret perf ;

        Verb2 : Type = Verb ** {c : Str} ;
    
        be_Verb : Verb = mkVerb "vara" "är" "var" "varit" ;
        
}

Abstract changed, previous concretes discarded.


In [9]:
concrete MicroLangSwe of MicroLang = open MicroResSwe in {
    lincat
    Utt = {s : Str} ;
    
    S = {s : Str} ;
    VP = {verb : Verb ; compl : Str} ;
    Comp = {s : Number => Species => Gender => Str} ;
    AP = Adjective ;
    CN = Noun ;
    NP = {s : Str} ;
    Det = {s : Gender => Str ; n : Number ; sp : Species} ;
    V = Verb ;
    V2 = Verb2 ;
    A = Adjective ; 
    N = Noun ;
    
    lin
-- Phrase
    UttS s = s ;
    UttNP np = {s = np.s} ;

-- Sentence
    PredVPS np vp = {
        s = np.s ++ vp.verb.s ! Pres
    } ;

-- Verb
    UseV v = {
        verb = v ;
        compl = [] ;
    } ;
    
    ComplV2 v2 np = {
        verb = v2 ;
        compl = v2.c ++ np.s
    } ;
    
    UseComp comp = {
        verb = be_Verb ;
        compl = []
    } ;
    
    CompAP ap = ap ;
    
-- Noun
    DetCN det cn = {
        s = det.s ! cn.g ++ cn.s ! det.n ! det.sp ;
    } ;

    a_Det = {
        s = table {
            Utr => "en" ;
            Neutr => "ett"
        } ;
        n = Sg ; 
        sp = Indef
    } ;
    aPl_Det = {
        s = table { _ => "flera" } ; 
        n = Pl ;
        sp = Indef
    } ;
    the_Det = {
        s = table {
            Utr => "den" ;
            Neutr => "det"
        } ;
        n = Sg ; 
        sp = Def
    } ;
    thePl_Det = {
        s = table { _ => "de" } ;
        n = Pl ;
        sp = Def
    } ;
    
    UseN n = n ;
    
    AdjCN ap cn = {
        g = cn.g ;
        s = table {
            n => table {
                sp => ap.s ! n ! sp ! cn.g ++ cn.s ! n ! sp
            }
        } ;
    } ;
    
-- Adjective
    PositA a = a ;

-- Lexicon part
    lin boy_N = mkN "pojke" Utr ;
    lin girl_N = mkN "flicka" Utr ;
 --   lin house_N = mkN "hus" Neutr ;
 --   lin clever_A = mkA "smart" ;
 --   lin big_A = mkA "stor" ;
 --   lin jump_V = mkV "hoppa" ;
 --   lin play_V = mkV "leka" "leker" "lekte" "lekt" ;
 --   lin love_V2 = mkV2 "älska" ;
    lin see_V2 = mkV2 (mkV "se" "ser" "såg" "sett") ;

-- Paradigms part
    oper 
    mkN = overload {
        mkN : Str -> Gender -> Noun   -- predictable noun, e.g. bil-bilen-bilar-bilarna, fluga-flugan-flugor-flugorna
          = \n,g -> lin N (smartNoun n g) ;
        mkN : Str -> Str -> Str -> Str -> Gender -> Noun  -- irregular noun, e.g. man-mannen-män-männen,
          = \sgIndef,sgDef,plIndef,plDef,gen -> lin N (mkNoun sgIndef sgDef plIndef plDef gen) ;
        } ;
    -- utrSgIndef, neutrSgIndef, sgDef, pl 
    mkA = overload {
        mkA : Str -> A -- predictable adjective
          = \adj -> lin A (smartAdjective adj) ;
        mkA : Str -> Str -> Str -> Str -> A -- very irregular adjective
          = \utrSgIndef,neutrSgIndef,sgDef,pl -> lin A (mkAdjective utrSgIndef neutrSgIndef sgDef pl) ;
    } ;
    
    
    mkV = overload {
        mkV : (inf : Str) -> V  -- predictable verb, e.g. baka-bakar-bakade-bakat, titta-tittar-tittade-tittat
            = \s -> lin V (regVerb s) ;
        mkV : (inf,pres,pret,perf : Str) -> V  -- irregular verb, e.g. sjunga-sjunger-sjöng-sjungit
            = \inf,pres,pret,perf -> lin V (irregVerb inf pres pret perf) ;
    } ;
  
    mkV2 = overload {
        mkV2 : Str -> V2          -- predictable verb with direct object, e.g. "hitta"
            = \s   -> lin V2 (regVerb s ** {c = []}) ;
        mkV2 : Str  -> Str -> V2  -- predictable verb with preposition, e.g. "vänta - på"
          = \s,p -> lin V2 (regVerb s ** {c = p}) ;
        mkV2 : V -> V2            -- any verb with direct object, e.g. "drink"
          = \v   -> lin V2 (v ** {c = []}) ;
        mkV2 : V -> Str -> V2     -- any verb with preposition
          = \v,p -> lin V2 (v ** {c = p}) ;
    } ;
}


Abstract changed, previous concretes discarded.


In [10]:
gr

PredVPS (DetCN aPl_Det (UseN girl_N)) (ComplV2 see_V2 (DetCN aPl_Det (UseN boy_N)))


In [11]:
gr | lin

flera pojkar ser


In [12]:
gt | lin

flera pojkar ser
flera pojkar ser
flera pojkar ser
flera pojkar ser
flera pojkar ser
flera pojkar ser
flera pojkar ser
flera pojkar ser
flera flickor ser
flera flickor ser
flera flickor ser
flera flickor ser
flera flickor ser
flera flickor ser
flera flickor ser
flera flickor ser
en pojke ser
en pojke ser
en pojke ser
en pojke ser
en pojke ser
en pojke ser
en pojke ser
en pojke ser
en flicka ser
en flicka ser
en flicka ser
en flicka ser
en flicka ser
en flicka ser
en flicka ser
en flicka ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de pojkarna ser
de flickorna ser
de flickorna ser
de flickorna ser
de flickorna ser
de flickorna ser
de flickorna ser
de flickorna ser
de flickorna ser
den pojken ser
den pojken ser
den pojken ser
den pojken ser
den pojken ser
den pojken ser
den pojken ser
den pojken ser
den flickan ser
den flickan ser
den flickan ser
den flickan ser
den flickan ser
den flickan ser
den flickan ser
den flic

In [14]:
export MicroLang
export MicroLangSwe
export MicroResSwe

Exported MicroLang.gf

Exported MicroLangSwe.gf

Exported MicroResSwe.gf